#### if initial env : on cmdline 
pip install -U langchain langchain_experimental openai

In [1]:
from langchain.pydantic_v1 import BaseModel
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [2]:
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import (
    create_openai_data_generator,
    OPENAI_TEMPLATE,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_SUFFIX,
    SYNTHETIC_FEW_SHOT_PREFIX,
)

In [6]:
import uuid

print(type(uuid.uuid4()))

<class 'uuid.UUID'>


In [14]:
#pydanticinit
class WelFareDataset(BaseModel) :
    query: uuid.UUID

#uuid test
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=WelFareDataset)

prompt = PromptTemplate(
    template="From this uuid, give me another uuid.\n{format_instructions}\n{uuid_input}",
    input_variables=["uuid_input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    )

### test run
_input = prompt.format_prompt(uuid_input=uuid.uuid4())

_input.to_string()

#일단 잘 됨. 근데 이걸로 데이터셋 어떻게 만듬?(진짜 모름)

'From this uuid, give me another uuid.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"query": {"title": "Query", "type": "string", "format": "uuid"}}, "required": ["query"]}\n```\n1028452d-564e-44d6-a037-1f121e85c472'

In [ ]:
##### DirectoryLoader 사용해서 구현한다음에 split 잘 되면 uuid 부여하기

In [15]:
from langchain.document_loaders import UnstructuredMarkdownLoader

In [18]:
test_document = UnstructuredMarkdownLoader(file_path="./markdownDB/01_생계_지원/01_LPG용기_사용가구_시설개선.md")

In [24]:
test_doc = test_document.load()
test_doc[0]

Document(page_content='LPG용기 사용가구 시설개선\n\n대상\n\nLPG용기 사용 주택에서 LPG 고무호스를 사용 중인 가구\n\n내용\n\nLP가스 고무호스 교체(금속배관) 및 안전장치(퓨즈콕) 등 가스시설 설치 지원 (시공비 약 25만 원 중 20만 원 상당 지원, 자부담 5만 원)\n\n방법\n\n수혜대상자가 소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터) 방문 신청\n\n문의\n\n소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터)\n\n한국가스안전공사(☎1544-4500)', metadata={'source': './markdownDB/01_생계_지원/01_LPG용기_사용가구_시설개선.md'})

In [25]:
test_doc[0].page_content

'LPG용기 사용가구 시설개선\n\n대상\n\nLPG용기 사용 주택에서 LPG 고무호스를 사용 중인 가구\n\n내용\n\nLP가스 고무호스 교체(금속배관) 및 안전장치(퓨즈콕) 등 가스시설 설치 지원 (시공비 약 25만 원 중 20만 원 상당 지원, 자부담 5만 원)\n\n방법\n\n수혜대상자가 소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터) 방문 신청\n\n문의\n\n소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터)\n\n한국가스안전공사(☎1544-4500)'

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators= "\n\n",
    chunk_size=200,
    is_separator_regex=False,
)

result = text_splitter.split_documents(test_doc)
result

[Document(page_content='LPG용기 사용가구 시설개선\n\n대상\n\nLPG용기 사용 주택에서 LPG 고무호스를 사용 중인 가구\n\n내용\n\nLP가스 고무호스 교체(금속배관) 및 안전장치(퓨즈콕) 등 가스시설 설치 지원 (시공비 약 25만 원 중 20만 원 상당 지원, 자부담 5만 원)\n\n방법\n\n수혜대상자가 소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터) 방문 신청\n\n문의', metadata={'source': './markdownDB/01_생계_지원/01_LPG용기_사용가구_시설개선.md'}),
 Document(page_content='내용\n\nLP가스 고무호스 교체(금속배관) 및 안전장치(퓨즈콕) 등 가스시설 설치 지원 (시공비 약 25만 원 중 20만 원 상당 지원, 자부담 5만 원)\n\n방법\n\n수혜대상자가 소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터) 방문 신청\n\n문의\n\n소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터)', metadata={'source': './markdownDB/01_생계_지원/01_LPG용기_사용가구_시설개선.md'}),
 Document(page_content='LP가스 고무호스 교체(금속배관) 및 안전장치(퓨즈콕) 등 가스시설 설치 지원 (시공비 약 25만 원 중 20만 원 상당 지원, 자부담 5만 원)\n\n방법\n\n수혜대상자가 소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터) 방문 신청\n\n문의\n\n소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터)\n\n한국가스안전공사(☎1544-4500)', metadata={'source': './markdownDB/01_생계_지원/01_LPG용기_사용가구_시설개선.md'})]